<img src='https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png' alt="SUT logo" width=200 height=200 align=left class="saturate" >

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Foundations of Data Science <br>
<font color=2565AE size=5>
    Electrical Engineering Department <br>
<font color=3C99D size=5>
    Project: Financial Analysis on Twitter <br>
<font color=696880 size=4>
    Parishad Mokhber - 98100537 <br>
    
____

# Bonus Part: Web Crawling

In [17]:
!pip install selenium beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 83.1 kB/s eta 0:00:0000:0100:04m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 155.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 58.3 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.6.3
    Uninstalling typing_extensions-4.6.3:
      Successfully uninstalled typing_extensions-4.6.3

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

options = webdriver.EdgeOptions()
driver = webdriver.Edge(options=options)

driver.get("https://www.digikala.com/search/notebook-netbook-ultrabook/")

SCROLL_PAUSE_TIME = 2  
screen_height = driver.execute_script("return window.screen.height;") 
i = 1
while True:
    driver.execute_script(f"window.scrollTo(0, {screen_height * i});")
    i += 1
    time.sleep(SCROLL_PAUSE_TIME)

    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    if screen_height * i > scroll_height:
        break

soup = BeautifulSoup(driver.page_source, 'html.parser')

ultrabooks = soup.find_all('div', class_='product-list_ProductList__item__LiiNI')

driver.quit()

In [6]:
ultrabooks

[<div class="product-list_ProductList__item__LiiNI" data-product-index="1"><a class="block cursor-pointer relative bg-neutral-000 overflow-hidden grow py-3 px-4 lg:px-2 h-full styles_VerticalProductCard--hover__ud7aD" href="/product/dkp-12537925/%D9%84%D9%BE-%D8%AA%D8%A7%D9%BE-156-%D8%A7%DB%8C%D9%86%DA%86%DB%8C-%D9%84%D9%86%D9%88%D9%88-%D9%85%D8%AF%D9%84-ideapad-1-celeron-4gb-256ssd/" target="_blank"><div class="h-full" data-testid="product-card"><article class="overflow-hidden flex flex-col items-stretch justify-start h-full"><div class="flex items-center justify-start mb-1"><div class="text-body2-extra grow"><br/></div></div><div class="flex grow relative flex-col"><div class=""><div class="flex items-stretch flex-col relative mb-1"><div class="flex items-start mx-auto"><div><div style="width: 240px; height: 240px; line-height: 0;"><picture><source srcset="https://dkstatics-public.digikala.com/digikala-products/e5dfffed0fe58135b9787887394f4db17be8c5dc_1693397601.jpg?x-oss-process=ima

In [47]:
df = pd.DataFrame(columns=['Text', 'Price'])

for i, ultrabook in enumerate(ultrabooks):
    text = ultrabook.find('h3', class_='styles_VerticalProductCard__productTitle__6zjjN')
    price = ultrabook.find('span', attrs={'data-testid': 'price-final'})
    
    if text != None:
        text = text.text.strip()
        
    if price != None:
        price = price.text.strip()
    
    if text != None and price != None:
        df.loc[i] = {'Text':text, 'Price':price}

In [48]:
df.head()

,Text,Price
0,لپ تاپ 15.6 اینچی لنوو مدل Ideapad 1-Celeron N...,"۹,۱۳۹,۰۰۰"
1,لپ تاپ 15.6 اینچی لنوو مدل IdeaPad Slim 3 15IR...,"۱۶,۴۸۰,۰۰۰"
2,لپ تاپ 15.6 اینچی ایسوس مدل X1500EP-EJ005W i5 ...,"۳۱,۵۰۰,۰۰۰"
3,لپ تاپ 15.6 اینچی ایسوس مدل Vivobook 15 R1502Z...,"۲۵,۴۹۹,۰۰۰"
4,لپ تاپ 15.6 اینچی لنوو مدل V15 G4 AMN-A 8GB 1S...,"۱۴,۱۵۰,۰۰۰"


In [50]:
import re

# Brand mapping
brand_mapping = {
    'مایکروسافت': 'Microsoft',
    'لنوو': 'Lenovo',
    'اچ‌پی': 'HP',
    'اپل': 'Apple',
    'ایسوس': 'Asus',
    'ای دیتا': 'AData',
}

# Regex patterns
screen_size_pattern = r'(\d{1,2}\.\d|\d{1,2}) اینچی'
model_pattern = r'مدل ([\w\s-]+)'
cpu_pattern = r'\b(i[3579]|Celeron|Pentium|AMN-A)\b'
ram_pattern = r'(\d+GB)'
ssd_pattern = r'(\d+SS)'

# Function to extract information
def extract_info(row):
    title = row['Text']

    # Extract brand
    brand = 'NA'
    for key in brand_mapping:
        if key in title:
            brand = brand_mapping[key]
            break

    # Extract other information
    screen_size = re.search(screen_size_pattern, title)
    model = re.search(model_pattern, title)
    cpu = re.search(cpu_pattern, title)
    ram = re.search(ram_pattern, title)
    ssd = re.search(ssd_pattern, title)

    return pd.Series({
        'Brand': brand,
        'Screen Size': screen_size.group(1) if screen_size else 'NA',
        'Model': model.group(1).split()[0] if model else 'NA',
        'CPU': cpu.group() if cpu else 'NA',
        'RAM': ram.group() if ram else 'NA',
        'SSD': ssd.group()[:-2] if ssd else 'NA'
    })

# Apply the extraction function
df[['Brand', 'Screen Size', 'Model', 'CPU', 'RAM', 'SSD']] = df.apply(extract_info, axis=1)

df.head()

,Text,Price,Brand,Screen Size,Model,CPU,RAM,SSD
0,لپ تاپ 15.6 اینچی لنوو مدل Ideapad 1-Celeron N...,"۹,۱۳۹,۰۰۰",Lenovo,15.6,Ideapad,Celeron,4GB,256
1,لپ تاپ 15.6 اینچی لنوو مدل IdeaPad Slim 3 15IR...,"۱۶,۴۸۰,۰۰۰",Lenovo,15.6,IdeaPad,i3,8GB,256
2,لپ تاپ 15.6 اینچی ایسوس مدل X1500EP-EJ005W i5 ...,"۳۱,۵۰۰,۰۰۰",Asus,15.6,X1500EP-EJ005W,i5,12GB,512
3,لپ تاپ 15.6 اینچی ایسوس مدل Vivobook 15 R1502Z...,"۲۵,۴۹۹,۰۰۰",Asus,15.6,Vivobook,i5,16GB,512
4,لپ تاپ 15.6 اینچی لنوو مدل V15 G4 AMN-A 8GB 1S...,"۱۴,۱۵۰,۰۰۰",Lenovo,15.6,V15,AMN-A,8GB,1


In [52]:
persian_to_english = {
    '۰': '0', '۱': '1', '۲': '2', '۳': '3', '۴': '4',
    '۵': '5', '۶': '6', '۷': '7', '۸': '8', '۹': '9',
}

# Function to convert Persian numbers to English
def convert_persian_to_english(persian_number):
    return ''.join(persian_to_english.get(ch, ch) for ch in persian_number)

# Apply conversion to the price column
df['Price'] = df['Price'].apply(convert_persian_to_english)

df.head()

,Text,Price,Brand,Screen Size,Model,CPU,RAM,SSD
0,لپ تاپ 15.6 اینچی لنوو مدل Ideapad 1-Celeron N...,"9,139,000",Lenovo,15.6,Ideapad,Celeron,4GB,256
1,لپ تاپ 15.6 اینچی لنوو مدل IdeaPad Slim 3 15IR...,"16,480,000",Lenovo,15.6,IdeaPad,i3,8GB,256
2,لپ تاپ 15.6 اینچی ایسوس مدل X1500EP-EJ005W i5 ...,"31,500,000",Asus,15.6,X1500EP-EJ005W,i5,12GB,512
3,لپ تاپ 15.6 اینچی ایسوس مدل Vivobook 15 R1502Z...,"25,499,000",Asus,15.6,Vivobook,i5,16GB,512
4,لپ تاپ 15.6 اینچی لنوو مدل V15 G4 AMN-A 8GB 1S...,"14,150,000",Lenovo,15.6,V15,AMN-A,8GB,1


In [53]:
df.tail()

,Text,Price,Brand,Screen Size,Model,CPU,RAM,SSD
115,لپ تاپ 15.6 اینچی لنوو مدل V15 G2 ITL-i5 8GB 1...,"27,249,000",Lenovo,15.6,V15,i5,8GB,NA
116,لپ تاپ 15.6 اینچی لنوو مدل V15 IGL-Celeron N40...,"9,900,000",Lenovo,15.6,V15,Celeron,4GB,256
117,لپ تاپ 15.6 اینچی ایسوس مدل Vivobook 15 M1502Q...,"28,000,000",Asus,15.6,Vivobook,NA,16GB,512
118,لپ تاپ 16 اینچی ایسوس مدل ROG Strix G16 G614JV...,"80,117,840",Asus,16,ROG,NA,NA,NA
119,لپ تاپ 16 اینچی لنوو مدل Legion Pro 5 16IRX8 i...,"85,000,000",Lenovo,16,Legion,i7,16GB,1


In [55]:
df.to_csv('Digikala_Laptops_Ultrabooks.csv')

# Discussion

We needed <code>BeautifulSoup</code> and <code>Selenium</code> libraries to perform the web scraper for ***Digikala***.

As you can see, this scipt can scroll the page to load more content and also able to extract each of the items contetns thoroughly, such as its <code>Price</code>, <code>Brand</code>, <code>Model</code>, <code>CPU</code>, <code>Screen Size</code>, <code>RAM</code> and its possible <code>SSD</code>.

We have saved the results as a CSV file named <code>Digikala_Laptops_Ultrabooks.csv</code>